In [6]:
!pip install transformers qwen_vl_utils accelerate -U
!pip install -U bitsandbytes
!apt install poppler-utils  # for PDF to image conversion
!pip install pdf2image


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [2]:
!pip install transformers qwen_vl_utils accelerate>=0.26.0 PEFT -U

In [3]:
!pip install -U bitsandbytes

In [7]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import torch

model_name = "NAMAA-Space/Qari-OCR-0.2.2.1-VL-2B-Instruct"


model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_name)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  21%|##        | 640M/3.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/116M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [1]:
from pdf2image import pdfinfo_from_path, convert_from_path

# PDF path
pdf_path = "/content/serah00346.pdf"

# Get the number of pages
info = pdfinfo_from_path(pdf_path)
num_pages = info["Pages"]
print(f"Total pages: {num_pages}")

image_paths = []

# Process each page one at a time to avoid memory crash
for i in range(1, num_pages + 1):
    print(f"Converting page {i}...")
    page_images = convert_from_path(pdf_path, dpi=200, first_page=i, last_page=i)
    img_path = f"/content/page_{i}.png"
    page_images[0].save(img_path, "PNG")
    image_paths.append(img_path)


Total pages: 158
Converting page 1...
Converting page 2...
Converting page 3...
Converting page 4...
Converting page 5...
Converting page 6...
Converting page 7...
Converting page 8...
Converting page 9...
Converting page 10...
Converting page 11...
Converting page 12...
Converting page 13...
Converting page 14...
Converting page 15...
Converting page 16...
Converting page 17...
Converting page 18...
Converting page 19...
Converting page 20...
Converting page 21...
Converting page 22...
Converting page 23...
Converting page 24...
Converting page 25...
Converting page 26...
Converting page 27...
Converting page 28...
Converting page 29...
Converting page 30...
Converting page 31...
Converting page 32...
Converting page 33...
Converting page 34...
Converting page 35...
Converting page 36...
Converting page 37...
Converting page 38...
Converting page 39...
Converting page 40...
Converting page 41...
Converting page 42...
Converting page 43...
Converting page 44...
Converting page 45...
Co

In [2]:
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import os


In [3]:
model_name = "NAMAA-Space/Qari-OCR-0.2.2.1-VL-2B-Instruct"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_name)
max_tokens = 2000


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
ocr_results = []

prompt = "الصورة تحتوي على صفحة من كتاب مكتوب باللغة العربية. أرجو استخراج النص بدقة كما هو ظاهر في الصورة دون أي إضافات أو تلخيص أو تفسير. فقط النص الكامل كما تقرأه."

for page_path in image_paths:
    print(f"Processing: {page_path}")
    image = Image.open(page_path)

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": prompt},
            ],
        }
    ]

    text_prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
    text=[text_prompt],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
      ).to("cpu")

    model.to("cpu")

    generated_ids = model.generate(**inputs, max_new_tokens=max_tokens)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]

    ocr_results.append(output_text)


Processing: /content/page_1.png


In [ ]:
from pdf2image import convert_from_path

# Path to your PDF file
pdf_path = "/content/serah00346.pdf"  # ⬅️ change this if needed

# Convert each page to image (DPI 300 for good quality)
images = convert_from_path(pdf_path, dpi=300)

# Save images to disk (optional, for inspection)
for idx, img in enumerate(images):
    img.save(f"/content/page_{idx + 1}.png", "PNG")

# Store paths
image_paths = [f"/content/page_{i + 1}.png" for i in range(len(images))]


In [ ]:

src="image.png"

prompt = "Below is the image of one page of a document, as well as some raw textual content that was previously extracted for it. Just return the plain text representation of this document as if you were reading it naturally. Do not hallucinate."
image.save(src)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": f"file://{src}"},
            {"type": "text", "text": prompt},
        ],
    }
]
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")
generated_ids = model.generate(**inputs, max_new_tokens=max_tokens)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]
os.remove(src)
print(output_text)


يتم الخبراء العسكريون الأمريكيون تقييم الموقع بعد عبور المصريين قناة السويس وانتهاء فعالية خط بارليف، وكانت نصيحة امريكا أن تحاول إسرائيل بكل جهد تخطى رعوس الكبارى المصرية خلال الساعات الأولى من نهار السابع من أكتوبر، وأن تقوم بتوجيه ضرية قوية لشبكة الصواريخ سام ١٥، وأن تتجنب القتال المباشر». والحقيقة أن هذا بالضبط ما نفذته إسرائيل يوم ٧ أكتوبر ـ اليوم الثاني للعمليات ـ بعد أن استمعت إلى «النصيحة الأمريكية» وكلمة النصيحة هنا مستمارة استخدامها اليمازار حتى يتفادى كلمة «خطة أمريكية». وفى الواقع فإنها خطة عسكرية أمريكية تشتمل أمرين لتحقيق هدفين عسكريين هما: الأول: هجوم مضاد إسرائيل لتحطيم رعوس الكبارى المصرية. الثاني: توجية ضرية جوية إسرائيلية ضد شبكة الدفاع الجوى المصري لثل فماليتها. ومن الملاحظ أن الخطة العسكرية الأمريكية المقترحة لإسرائيل، تتمشى تمامًا مع التخطيط السياسي الأمريكي الذي بنى فى هذه المرحلة من الحرب على أساس أن القتال سيتطور بسرعة لصالح إسرائيل التي تُعيد الموقف إلى ما كان عليه فى وقت قصير. وبتعببير كسنجر «إستراتيجييتا الخاصة: انتظار إسرائيل لاستعادة وضها العسكري». وفى الص